In [1]:
import sys
sys.path.append("lag-llama")
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset
from lag_llama.gluon.estimator import LagLlamaEstimator
from gluonts.evaluation import make_evaluation_predictions
from gluonts.dataset.pandas import PandasDataset
import os
import plotly.graph_objects as go
from gluonts.torch.distributions import StudentTOutput
from gluonts.torch.modules.loss import NegativeLogLikelihood

c:\Users\latip\Desktop\Курс\3. Проектная работа\Проба\Сайт\env\lib\site-packages\gluonts\json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [2]:
torch.serialization.add_safe_globals([StudentTOutput, NegativeLogLikelihood, set])

In [3]:
# Загрузка данных из Excel файла
def load_single_excel_data(file_path):
    df = pd.read_excel(file_path, header=0)
    current_values = df.iloc[:, 1].values.astype(float)  # Берем только второй столбец
    return current_values


In [4]:
# Подготовка датасета
data = load_single_excel_data('data.xlsx')
scaler = MinMaxScaler()
data = data.reshape(-1, 1)  # Изменяем форму для нормализации
data = scaler.fit_transform(data).flatten().astype(np.float32)  # Приведение к типу float32
df = pd.DataFrame({
    "item_id": ["series_1"] * len(data),  # Добавляем столбец item_id
    "target": data
})
dataset = PandasDataset.from_long_dataframe(df, target="target", item_id="item_id")

In [5]:
df = pd.DataFrame({
    "item_id": ["series_1"] * len(data),  # Добавляем столбец item_id
    "target": data
})

In [6]:
dataset = PandasDataset.from_long_dataframe(df, target="target", item_id="item_id")

In [7]:
import os

In [8]:
# Определение функции предсказания с использованием Lag-Llama
def get_lag_llama_predictions(dataset, prediction_length, device, context_length=32, use_rope_scaling=False, num_samples=100):
    # Определяем путь к файлу lag-llama.ckpt
    ckpt_path = os.path.join("lag-llama", "lag-llama.ckpt")

    # Загружаем модель с использованием weights_only=True
    ckpt = torch.load(ckpt_path, map_location=device, weights_only=True)
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    rope_scaling_arguments = {
        "type": "linear",
        "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
    }

    estimator = LagLlamaEstimator(
        ckpt_path=ckpt_path,
        prediction_length=prediction_length,
        context_length=context_length,
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],
        rope_scaling=rope_scaling_arguments if use_rope_scaling else None,
        batch_size=1,
        num_parallel_samples=num_samples,
        device=device,
    )

    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)

    return forecasts, tss

In [9]:
# Настройки для предсказания
prediction_length = 24  # Длина прогноза
num_samples = 100  # Количество сэмплов для каждого временного шага
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Используем GPU, если доступен


In [10]:
# Получение предсказаний
forecasts, tss = get_lag_llama_predictions(dataset, prediction_length, device, num_samples=num_samples)


c:\Users\latip\Desktop\Курс\3. Проектная работа\Проба\Сайт\env\lib\site-packages\lightning\fabric\utilities\cloud_io.py:56: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
c:\Users\latip\Deskt

In [11]:
# Визуализация результатов с использованием Plotly
# Создаем фигуру
fig = go.Figure()

# Индекс последней точки оригинального ряда
last_index = len(data) - 1

# Отображаем истинные значения
fig.add_trace(go.Scatter(x=np.arange(last_index + 1), y=data, mode='lines', name='Истинные значения', line=dict(color='blue')))

# Создаем массив для предсказанных значений, начиная с последней точки оригинального ряда
predicted_values = [data[last_index]] + [forecast.samples[0][i] for forecast in forecasts for i in range(len(forecast.samples[0]))]

# Создаем массив для индексов прогноза
forecast_index = np.arange(last_index, last_index + prediction_length + 1)

# Отображаем прогнозы
fig.add_trace(go.Scatter(x=forecast_index, y=predicted_values, mode='lines', name='Прогноз', line=dict(color='green')))

# Настройки отображения
fig.update_layout(title="Прогноз временного ряда", xaxis_title="Индекс", yaxis_title="Значение", showlegend=True)

# Показываем график
fig.show()